## Make Isso HTTPS in TLJH

In [ ]:
%%bash
/opt/tljh/hub/bin/traefik version

* https://github.com/jupyterhub/the-littlest-jupyterhub/blob/master/tljh/jupyterhub_config.py
* https://github.com/jupyterhub/traefik-proxy/blob/master/jupyterhub_traefik_proxy/toml.py
* https://jupyterhub-traefik-proxy.readthedocs.io/en/latest/toml.html#traefik-configuration
* https://docs.traefik.io/v1.7/basics/
* https://posativ.org/isso/docs/setup/sub-uri/

for traefik v1.7

In [ ]:
%%bash
cat << EOF | sudo tee -a /opt/tljh/state/rules.toml > /dev/null

[frontends.isso]
backend = "isso"
passHostHeader = true

[frontends.isso.routes.f1]
rule = "PathPrefixStrip:/isso/"

[backends.isso.servers.s1]
url = "http://127.0.0.1:8000"
weight = 1
EOF

In [ ]:
%env http=http:\/\/jhub.name:8000\/
%env https=https:\/\/jhub.name\/isso\/
!sed -i -e "s/$http/$https/g" ~/hexo/themes/freemind/layout/_partial/post/comment.ejs

for traefik v2.10 [StripPrefix](https://doc.traefik.io/traefik/v2.10/middlewares/http/stripprefix/)

In [1]:
%%bash
cat << EOF | sudo tee /opt/tljh/state/rules/isso.toml > /dev/null
[http.routers.isso]
service = "isso"
rule = "PathPrefix(\`/isso/\`)"
entryPoints = [ "https",]
middlewares = ["isso"]

[http.middlewares]
  [http.middlewares.isso.stripPrefix]
    prefixes = ["/isso"]

[http.services.isso.loadBalancer]
passHostHeader = true
[[http.services.isso.loadBalancer.servers]]
url = "http://127.0.0.1:8000"
EOF

## Commit Modifications
https://github.com/seii-saintway/hexo-theme-freemind/commit/9568c3160d3c917a8a986beedafb7836e02c2bce

### Change file-watch to directory in TLJH Traefik
https://github.com/seii-saintway/the-littlest-jupyterhub/commit/e95d85e99187f9d65b748016807b28c256e77e39

In [ ]:
%%bash
cat << EOF | sudo tee /opt/tljh/state/dynamic/isso.toml > /dev/null
[frontends.isso]
backend = "isso"
passHostHeader = true

[frontends.isso.routes.f1]
rule = "PathPrefixStrip:/isso/"

[backends.isso.servers.s1]
url = "http://127.0.0.1:8000"
weight = 1
EOF

In [ ]:
!sudo /opt/tljh/hub/bin/python -m tljh.installer

Granting passwordless sudo to JupyterHub admins...
Setting up user environment...


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.8
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda


Setting up JupyterHub...


In [ ]:
!sudo cat /opt/tljh/installer.log

2019-10-13 15:21:02,889 Granting passwordless sudo to JupyterHub admins...
2019-10-13 15:21:02,890 Setting up user environment...
2019-10-13 15:21:26,052 Setting up JupyterHub...


In [ ]:
!cd ~ && make restart-isso

## Isso SQLite

In [ ]:
%%bash
cat << EOF | sudo tee /etc/isso.conf > /dev/null

[general]

dbpath = /var/lib/isso/comments.db

host =
    http://seii-saintway.github.io
    https://seii-saintway.github.io
    http://digitypography.xyz
    https://digitypography.xyz

EOF

### Open SQLite database
``` bash
sqlite3 /var/lib/isso/comments.db
```

In [ ]:
import sqlite3
conn = sqlite3.connect('/var/lib/isso/comments.db')

In [ ]:
[(row[6], row[7]) for row in conn.cursor().execute('SELECT * from comments')]

[('110.3.247.0', '# Privacy Policy'),
 ('113.38.215.0', '# Continuous Purification'),
 ('110.3.247.0',
  '### 3.11.4 隐秘通道\n\n隐秘通道（covert channel）的意思是说使用难以预料或不会引起注意的方式来发送信息。假设有一天系统管理员Sally觉得自己的那些用户花太多时间来玩了，因此她决定关闭email和即时消息，这样他们就不能聊天了。要解决这个问题，你和你的朋友都同意使用主目录中的一个你们都可以读取的文件来交流信息，你每隔一会儿就得检查一下该文件来查看新消息。这种超出乎预料的通信方式就是一个隐秘通道。\n\n隐秘通道很难取消。如果系统管理员Sally发现了你们这种基于文件的通信技术，她可以修改目录的权限，使得只有目录的属主才具有读取和搜索的权限，并限定目录属主也不能修改这种权限。在这样做的同时，她还可以确认你不能在其他地方（例如，/tmp）创建文件。（你的大部分程序都不能运行了，但是不要因此而责怪Sally。）即使是这样，你和你的朋友仍然可以相互显示其他用户士目录中的内容，这可以显示目录的修改日期和文件个数，这样你就可以根据这些可见的参数设计一种密码，并通过修改这些参数而进行通信。这是一种更复杂的隐秘通道，如果Sally对你们进行了更严格的限制，你们可以想出更古怪的法子来。\n\nSSH不能防止隐秘通道。对隐秘通道的分析和控制通常都是安全性很高的计算机系统的一部分，例如设计用来在同一个系统中的不同安全层次上安全地处理信息的系统。顺便说一下，SSH数据流本身就可以很好地用作一种隐秘通道：SSH会话的内容可能是一个巧克力馅饼的配方，而隐含的内容则可能是用Morse码表示这两个公司即将合并，只需要使用报文长度是奇数还是偶数来分别表示Morse码的短横线和点号即可。'),
 ('126.158.192.0', 'Try to build with Cosmos SDK'),
 ('126.158.192.0', 'Try to build with Cosmos SDK')]

In [ ]:
[row for row in conn.cursor().execute('SELECT * from preferences')]

In [ ]:
[row for row in conn.cursor().execute('SELECT * from threads')]

[(1, '/2019/07/17/%E5%BF%83%E6%83%85%E7%B3%9F%E9%80%8F%E4%BA%86/', None),
 (2, '/2022/02/25/Declaration-of-the-Independence-of-Cyberspace/', None),
 (3, '/2022/02/28/An-Independent-Stablecoin-of-Cyberspace/', None)]

In [ ]:
conn.close()